In [31]:
import os
import re
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [ ]:
def create_dataframe(data, columns):
    """
    Erstellt ein pandas DataFrame aus einer Liste von Listen und einer Liste von Spaltennamen.

    :param data: List[List], die Datenreihen
    :param columns: List[str], die Spaltennamen
    :return: pandas.DataFrame
    """
    return pd.DataFrame(data, columns=columns)

"""
ohne Sammelseiten:
* deutsche Film und Fernsehakademie
* deutsches Zentrum für Hochschul und Wissenschaftsforschung
* Berliner Werkstätten für Menschen mit Behinderung
"""
data = [
    # ['IBB', 'https://www.ibb.de/de/ueber-uns/finanzberichte/finanzberichte.html', 'Jahresabschluss', 'text', 'beautifulsoup'],
    # ['IBB', 'https://www.ibb.de/de/service/download-center/download-center.html?tag=03_geschaeftsberichte', 'Geschäftsbericht', 'text', 'beautifulsoup'],

    # ['degewo AG', 'https://www.degewo.de/presse', 'Konzernlagebericht', 'text', 'beautifulsoup'],
    # ['GESOBAU AG', 'https://www.gesobau.de/ueber-uns/gesobau-in-zahlen/geschaeftsbericht/', r'Gesch.{1,2}ftsbericht', 'url', 'beautifulsoup'],
    
    # ['Berlinovo', 'https://www.berlinovo.de/de/presse/geschaeftsbericht-2021', r'Geschäftsbericht|Jahresabschluss', 'text', 'beautifulsoup'],
    # ['Berlinovo', 'https://2023.berlinovo-geschäftsbericht.de/downloads', 'Finanzbericht', 'text', 'beautifulsoup'], # muss man wohl immer anpassen

    # ['Amt für Statistik Berlin-Brandenburg', 'https://www.statistik-berlin-brandenburg.de/geschaeftsberichte', 'Geschaeftsbericht', 'url', 'selenium'],

    # ['Berlin Energie und Netzholding', 'https://be-nh.de/dokumente/', 'Jahresabschluss', 'text', 'beautifulsoup'],

    ['Berliner Bäder Betriebe', 'https://www.berlinerbaeder.de/unternehmen/geschaeftsberichte/', r'Gesch.{1,2}ftsbericht', 'text', 'beautifulsoup'],
]

columns = ["Unternehmen", "URL", "Stichwort", 'Suchbereich', 'Funktion']

df = create_dataframe(data, columns)
print(df)

               Unternehmen                                                URL  \
0  Berliner Bäder Betriebe  https://www.berlinerbaeder.de/unternehmen/gesc...   

               Stichwort Suchbereich       Funktion  
0  Gesch.{1,2}ftsbericht        text  beautifulsoup  


In [33]:
def download_pdfs(url, DOWNLOAD_FOLDER, pattern, link_part = 'text', verbose = False):
    response = requests.get(url)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.text, "html.parser")
    links = soup.find_all("a", href=True)

    count_downloaded = 0
    count_skipped = 0
    count_error = 0
    
    for link in links:
        if link_part == 'text':
            search_string = link.text
        elif link_part == 'url':
            search_string = link['href']
        if re.search(pattern, search_string) and link["href"].endswith(".pdf"):
            pdf_url = link["href"]
            if not pdf_url.startswith("http"):
                pdf_url = requests.compat.urljoin(url, pdf_url)
            
            pdf_name = os.path.join(DOWNLOAD_FOLDER, pdf_url.split("/")[-1])

            if os.path.exists(pdf_name):
                if verbose:
                    print(f"Bereits vorhanden: {pdf_name}")
                
                count_skipped += 1
                continue
            
            if verbose:
                print(f"Lade herunter: {pdf_url}")

            pdf_response = requests.get(pdf_url)
            with open(pdf_name, "wb") as file:
                file.write(pdf_response.content)
                count_downloaded += 1
                
                if verbose:
                    print(f"Gespeichert als: {pdf_name}")

    if not verbose:
        print(f'Es wurden {count_downloaded} Berichte heruntergeladen.\nEs wurden {count_skipped} schon vorhandene Berichte übersprungen.\nEs gab {count_error} Fehler.')

In [90]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
import os
import requests

def download_pdfs_with_selenium(url, download_folder, pattern, link_part='text', wait_time=10, verbose=False):
    """
    Opens a website with Chromium using Selenium and downloads all PDF files where the link name or label matches a regex pattern.

    :param url: str, the URL of the website
    :param download_folder: str, the folder where PDFs will be saved
    :param pattern: str, the regex pattern to match link names or labels
    :param link_part: str, 'text' to match link text or 'url' to match href attribute
    :param wait_time: int, time to wait for the page to load
    :param verbose: bool, whether to print detailed logs
    """
    # Set up Selenium WebDriver (ensure you have the correct path to your chromedriver)
    # service = Service('path/to/chromedriver')  # Replace with the path to your chromedriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run in headless mode
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), 
        options=options
        )

    try:
        # Open the URL
        driver.get(url)

        count_downloaded = 0
        count_skipped = 0
        count_error = 0

        while True:
            # Wait for the page to load
            try:
                # Wait for the page to load
                WebDriverWait(driver, wait_time).until(
                    EC.presence_of_all_elements_located((
                        # By.TAG_NAME, 'a'
                        By.CLASS_NAME, 'afs-item-result'
                        ))
                )
            except TimeoutException:
                print("Timeout waiting for page to load.")
                break

            # Find all links on the page
            links = driver.find_elements(By.TAG_NAME, 'a')

            for link in links:
                try:           
                    if link_part == 'text':
                        search_string = link.text
                    elif link_part == 'url':
                        search_string = link.get_attribute('href')

                    if search_string and re.search(pattern, search_string) and search_string.endswith('.pdf'):
                        pdf_url = search_string if link_part == 'url' else link.get_attribute('href')
                        if not pdf_url.startswith("http"):
                            pdf_url = requests.compat.urljoin(url, pdf_url)

                        pdf_name = os.path.join(download_folder, pdf_url.split("/")[-1])

                        if os.path.exists(pdf_name):
                            if verbose:
                                print(f"Bereits vorhanden: {pdf_name}")
                            count_skipped += 1
                            continue

                        if verbose:
                            print(f"Lade herunter: {pdf_url}")

                        pdf_response = requests.get(pdf_url)
                        with open(pdf_name, "wb") as file:
                            file.write(pdf_response.content)
                            count_downloaded += 1

                            if verbose:
                                print(f"Gespeichert als: {pdf_name}")
                except Exception as e:
                    print(f"Fehler beim Abrufen des Links: {e}")
                    count_error += 1
                    continue

            # Check for the next page button
            # funktioniert für Amt für Statistik nicht
            try:
                # Locate the parent element with class 'pagination'
                pagination = driver.find_element(By.CLASS_NAME, 'pagination')
                # Find the last element with class 'pagination-nav-item' within the parent
                next_button = pagination.find_elements(By.CLASS_NAME, 'pagination-nav-item')[-1].find_element(By.TAG_NAME, 'a')
                print(next_button.label)
                
                if 'disabled' in next_button.get_attribute('class'):
                    print("No more pages to navigate.")
                    break
                next_button.click()
                time.sleep(2)  # Wait for the next page to load
            except Exception as e:
                print("No next page button found or error navigating to the next page.")
                break

        if not verbose:
            print(f'Es wurden {count_downloaded} Berichte heruntergeladen.\nEs wurden {count_skipped} schon vorhandene Berichte übersprungen.')

    finally:
        # Close the browser
        driver.quit()
        # pass

In [100]:
def pdf_downloader_loop(df, download_parent_folder = 'Geschaeftsberichte/'):
    for index, row in df.iterrows():
        unternehmen = row['Unternehmen']
        URL = row['URL']
        stichwort = row['Stichwort']
        suchbereich = row['Suchbereich']
        f = row['Funktion']

        # Verzeichnis für die heruntergeladenen PDFs
        DOWNLOAD_FOLDER = download_parent_folder+unternehmen
        verbose = False if os.path.exists(DOWNLOAD_FOLDER) else True
        os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)

        print(f'\n####### {unternehmen} #######')
        if f == 'beautifulsoup':
            download_pdfs(URL, DOWNLOAD_FOLDER, stichwort, suchbereich, verbose)
        elif f == 'selenium':
            download_pdfs_with_selenium(URL, DOWNLOAD_FOLDER, stichwort, suchbereich, verbose=verbose)
        file_count = len([file for file in os.listdir(DOWNLOAD_FOLDER) if file.endswith('.pdf')])
        print(f'Es befinden sich {file_count} Berichte im Verzeichnis.')

pdf_downloader_loop(df)


####### Berliner Bäder Betriebe #######
Lade herunter: https://www.berlinerbaeder.de/fileadmin/BBB/Dateien/Unternehmen/Geschaeftsberichte/2023_Geschaeftsbericht_BBB.pdf
Gespeichert als: Geschaeftsberichte/Berliner Bäder Betriebe/2023_Geschaeftsbericht_BBB.pdf
Lade herunter: https://www.berlinerbaeder.de/fileadmin/BBB/Dateien/Unternehmen/Geschaeftsberichte/2022_Geschaeftsbericht_BBB.pdf
Gespeichert als: Geschaeftsberichte/Berliner Bäder Betriebe/2022_Geschaeftsbericht_BBB.pdf
Lade herunter: https://www.berlinerbaeder.de/fileadmin/BBB/Dateien/Unternehmen/Geschaeftsberichte/2021_Geschaeftsbericht_BBB.pdf
Gespeichert als: Geschaeftsberichte/Berliner Bäder Betriebe/2021_Geschaeftsbericht_BBB.pdf
Lade herunter: https://www.berlinerbaeder.de/fileadmin/BBB/Dateien/Unternehmen/Geschaeftsberichte/2020_Geschaeftsbericht_BBB.pdf
Gespeichert als: Geschaeftsberichte/Berliner Bäder Betriebe/2020_Geschaeftsbericht_BBB.pdf
Lade herunter: https://www.berlinerbaeder.de/fileadmin/BBB/Dateien/Unternehmen/